# Pre processing - Standardization

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?code_challenge=U2-HsAQeCewCgfvJBY1nbYBYIZ-kl2zxq42l1T2SIq4&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


Enter verification code: 4/tgGlCIGyni9kIyUSgcrM3taIYJReZMI0UdSbmf537TAzMlTicwIWd0I
If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [galli.giuly@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [0]:
!gcloud config set project reddit-master

Updated property [core/project].


In [0]:
import pandas as pd
import string
import spacy 
import pickle
import ast

from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn import preprocessing
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split




In [0]:
!gsutil cp gs://reddit_final_results/comments_posts_2018_V2.csv .

Copying gs://reddit_final_results/comments_posts_2018_V2.csv...
| [1 files][  2.0 GiB/  2.0 GiB]   54.5 MiB/s                                   
Operation completed over 1 objects/2.0 GiB.                                      


In [0]:
comments_posts_df = pd.read_csv("comments_posts_2018_V2.csv", index_col=[0])

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
comments_posts_df.columns

Index(['subreddit', 'body'], dtype='object')

In [0]:
comments_posts_df.head(5)

,subreddit,body
0,aww,doubling down with the multiple sub approach g...
1,aww,i 2nd meatball i have a seriously chunky engli...
2,aww,my thoughts as well
3,aww,friends don't eat friends
4,aww,maybe the doctors are waiting with the heart i...


Checking if there are NaN values and delete them

In [0]:
comments_posts_df["body"].isna().sum()

32288

In [0]:
comments_posts_df = comments_posts_df.dropna()

## Encoding of the prediction target

In [0]:
# Create a column named subreddit_id 
comments_posts_df['subreddit_id'] = comments_posts_df['subreddit']


In [0]:
comments_posts_df.head(5)

,subreddit,body,subreddit_id
0,aww,doubling down with the multiple sub approach g...,aww
1,aww,i 2nd meatball i have a seriously chunky engli...,aww
2,aww,my thoughts as well,aww
3,aww,friends don't eat friends,aww
4,aww,maybe the doctors are waiting with the heart i...,aww


In [0]:
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [0]:
# Fit the encoder to the pandas column
le.fit(comments_posts_df['subreddit'])

LabelEncoder()

In [0]:
# View the labels (if you want)
list(le.classes_)

['Fitness',
 'IAmA',
 'atheism',
 'aww',
 'europe',
 'funny',
 'gaming',
 'movies',
 'nba',
 'politics',
 'science',
 'technology',
 'todayilearned',
 'worldnews']

In [0]:
# Apply the fitted encoder to the pandas column
comments_posts_df['subreddit_id'] = le.transform(comments_posts_df['subreddit']) 

In [0]:
comments_posts_df.head(5)

,subreddit,body,subreddit_id
0,aww,doubling down with the multiple sub approach g...,3
1,aww,i 2nd meatball i have a seriously chunky engli...,3
2,aww,my thoughts as well,3
3,aww,friends don't eat friends,3
4,aww,maybe the doctors are waiting with the heart i...,3


## Tokenization - stopwords - stemming

In [0]:
# nlp = spacy.load("en", disable=['parser', 'ner'])
# stemmer = SnowballStemmer(language='english')

In [0]:
stop_words = spacy.lang.en.stop_words.STOP_WORDS
print('Number of stopwords: %d' % len(stop_words))
print(list(stop_words))

Number of stopwords: 326
['four', 'mostly', 'is', 'get', 'nowhere', 'forty', 'hence', 'show', 'three', 'in', 'whether', 'above', 'any', 'last', 'due', "'ve", "n't", 'their', 'does', 'done', 'everywhere', 'that', 'being', 'eight', 'never', 'beyond', 'though', 'anyhow', 'indeed', 'none', 'nor', '’ll', 'whence', 'may', 'thereby', 'neither', 'side', 'which', 'myself', 'otherwise', 'ten', 'hereby', "'d", 'elsewhere', 'had', 'behind', 'throughout', 'must', 'too', 'by', '‘m', 'per', '‘d', 'his', 'should', 'latterly', 'you', 'before', 'whereas', 'becomes', 'wherever', 'some', 'two', 'within', 'no', 'whereupon', 'nine', 'twenty', 'for', 'further', 'if', 'what', 'even', 'are', 'so', 'say', 'take', 'towards', 'each', 'was', 'could', 'herself', 'seemed', 'a', 'can', 'just', 'do', 'meanwhile', 'name', 'empty', 'quite', 'really', 'these', 'here', 'often', 'n‘t', 'other', 'yours', 'afterwards', 'wherein', 'eleven', 'however', 'next', 'together', '’s', 'anywhere', 'such', 'became', 'between', 'move', 

In [0]:
parser = English()
stemmer = SnowballStemmer(language='english')

# tokenization
def tokenize(text):
    return parser(text)

# stopwords
def remove_stopwords(tokenized):
    without_stopwords = []

    for token in tokenized:
        if token.text not in stop_words:
            without_stopwords.append(token.text)

    return without_stopwords

# stemming
def stem(words):
    stemmed_words = []
    for word in words:
        stemmed_words.append(stemmer.stem(word))
    return stemmed_words


def pre_process (text):
    return stem(remove_stopwords(tokenize(text)))

def pre_process_df (df,column):
    df[column] = df[column].map(pre_process)
    return 

In [0]:
# testing the pre_process function on a simple string
sample = "tried try trying to tokenize our text doesn't"
pre_process(sample)

'tri tri tri token text'

In [0]:
# testing the pre_process function on a df
sample = comments_posts_df.sample(10)
pre_process_df(sample,'body')
sample

,subreddit,body,subreddit_id
4637645,technology,dishonest,11
7922006,Fitness,somewhat explain wrist wo complet straight l...,0
4302535,technology,yes fcc ration depart fcc head man elect his...,11
1169108,atheism,honest interest argu point time know taken a...,2
4881231,aww,gorgeous dog adventur okay anymor,3
1302173,science,think agreement forse wealthi nation have hand...,10
4027378,politics,unhing guy drink problem shadi polit oper rape...,9
9627335,atheism,religi suffer time express real suffer protest...,2
7815920,worldnews,econom cycl bare trump,13
2355839,europe,gtif 200k deport annuallyyou know near come eu...,4


In [0]:
# appling the pre_process function to our comments_posts_df (%3h)
pre_process_df(comments_posts_df,'body')

In [0]:
comments_posts_df.head()

In [0]:
!pwd

/content


In [0]:
comments_posts_df.to_csv('comments_posts_tokenized.csv')

The csv save the `body` columns as a string, what we need is to convert it as a **list**

In [0]:
comments_posts_tokenized_df['body'] = comments_posts_tokenized_df['body'].map(lambda x: ast.literal_eval(x))

In [0]:
# it worked

type(comments_posts_tokenized_df['body'].iloc[0])

list

In [0]:
# saving the final df with pickle

comments_posts_tokenized_df.to_pickle("comments_posts_tokenized_df.pkl", compression=gzip)

In [0]:
!ls

adc.json		      comments_posts_tokenized_df.pkl
comments_posts_tokenized.csv  sample_data


In [0]:
# storaging the final df into the bucket reddit_final_results

!gsutil cp /content/comments_posts_tokenized_df.pkl gs://reddit_final_results/

Copying file:///content/comments_posts_tokenized_df.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

|
Operation completed over 1 objects/1.6 GiB.                                      


In [0]:
!gsutil cp gs://reddit_final_results/comments_posts_tokenized_df.pkl .

Copying gs://reddit_final_results/comments_posts_tokenized_df.pkl...
| [1 files][  1.6 GiB/  1.6 GiB]   71.6 MiB/s                                   
Operation completed over 1 objects/1.6 GiB.                                      


In [0]:
!ls

adc.json  comments_posts_tokenized_df.pkl  sample_data


In [0]:
comments_posts_tokenized_df = pd.read_pickle("/content/comments_posts_tokenized_df.pkl")

In [0]:
comments_posts_tokenized_df.to_pickle("comments_posts_tokenized_df_gzip.pkl", compression="gzip")

In [0]:
# storaging the final df compressed into the bucket reddit_final_results

!gsutil cp /content/comments_posts_tokenized_df_gzip.pkl gs://reddit_final_results/

Copying file:///content/comments_posts_tokenized_df_gzip.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/
Operation completed over 1 objects/488.7 MiB.                                    
